In [ ]:
##########################################################################
#################### SETTINGS CELL, INITIALISES VARIABLES ################
##########################################################################
### EXECUTE THIS CELL FIRST

import os
import fnmatch
import csv
import pandas as pd
import json
import itertools
import random
from collections import Counter
######################
## Set variables #####
######################

csvDir = "C:\\PATH\\TO\\FILES\\"
#make sure you have a double backward slash at the end of the path!metadata = csvDir + "csv_metadata.txt"

# bias threshold for choosing partner name over the supplied list. 
# keep small to preserve 'mystery'
biasThreshold = 0.1

def getUserNames(aDict):
    uList = []
    for key, value in aDict.items():
        if(value['userDefined'] == 'y'):
            uList.append(key[:len(key)-4])
    return uList

## get the session name and remove ass
sessionName = input("Please enter your name: ")
print("Results will be appended to appropriate file. Names from this file (if exists) will not be offered")
sessionName = sessionName.lower()

In [ ]:
##########################################################################
################# PREPROCESSING CELL, CREATES NAMES ORDER ################
##########################################################################
## EXECUTE THIS CELL SECOND (2nd)


## First check the list of csvs to see if there is a matching metadata value 
fileList = fnmatch.filter(os.listdir(csvDir), '*.csv')

metaDict = {}
if os.path.exists(metadata):
    reader = csv.DictReader(open(metadata,'w'))
    for line in reader:
        metaDict[line['fileName']] = {'column': int(line['column']),
                                      'header': line['header'],
                                      'userDefined': line['userDefined']}
else:
     metaDict = {}
externalList = []
compareList = []
myList = []

for aFile in fileList:
    if aFile not in metaDict:
        print("\nMISSING METADATA FOR FILE: {}".format(aFile))
        with open(csvDir+'\\'+aFile,'r',encoding="iso8859_10") as wf:
            print("line 1: " + wf.readline().strip())
            print("line 2: " + wf.readline().strip())
            header = input("Is there a header (y/n)? ")
            if(header != 'y'):
                headerStatus = False   
            else:
                headerStatus = True   
            
            user = input("is this a user-supplied list (y/n)? ")
            col = 0
            if(user != 'y'):
                user = 'False'
                col = input("which column represents the name (starting at 0)? ")
            else:
                user = 'True'
            ## write the metadata out as an append statement, also add to the metaDict object
            if os.path.exists(metadata):
                with open(metadata,'a') as f:
                    f.write('\n'+aFile + ',' + str(col) + ',' + str(headerStatus) + ',' + str(user))
                    metaDict[aFile] = {'column':col,'header': str(headerStatus), 'userDefined':str(user)}
            else:
                with open(metadata,'a') as f:
                    f.write('fileName,column,header,userDefined\n')
                    f.write(aFile + ',' + str(col) + ',' + str(headerStatus) + ',' + str(user))
                    metaDict[aFile] = {'column':col,'header': str(headerStatus), 'userDefined':str(user)}
    
    with open(csvDir+'\\'+aFile, newline='\n',encoding="iso8859_10") as f:
        reader = csv.reader(f)
        if(metaDict[aFile]["header"] == 'True'):
            next(reader)
        data = list(reader)
        if(metaDict[aFile]["userDefined"] == 'True'):
            ## check if this is session user file. If it isn't. add the names to the compare list
            if(sessionName != aFile[:len(aFile)-4]):
                ## first check if the weighting is zero. No point comparing names you hated
                # if it is zero, add the name to the myList. Has the effect of removing the names from any future comparison
                # otherwise add to the compareList, which means it will be shown more often
                for row in data:
                    if(row[1] == '0'):
                        myList.append(row[0])
                    else:
                        compareList.append(row[0])
            else:
                # we are looking at the session user file. We don't want to see these names at all
                myList.extend([row[0] for row in data])
        else:
            externalList.extend([row[int(metaDict[aFile]['column'])] for row in data])

#remove duplicates from the externalList. 
# Only list expected to have dupes as the partner list and the personal list are cleaned before entry
externalList = list(dict.fromkeys(externalList))

# remove any names in the personal list from the the partner and external lists
externalList = list((Counter(externalList) - (Counter(myList) & Counter(externalList))).elements())
compareList = list((Counter(compareList) - (Counter(myList) & Counter(compareList))).elements())

# remove any names in the partner list from the external list 
# This removes dupes but ensuring they are more likely to turn up
externalList = list((Counter(externalList) - (Counter(compareList) & Counter(externalList))).elements())

# generate list of samples, in order
externalOrder = random.sample(range(len(externalList)), k=len(externalList))
compareOrder = random.sample(range(len(compareList)), k=len(compareList))
externalPosition = 0
comparePosition = 0

   

In [ ]:
##########################################################################
########### THE MAGIC CELL, WHERE THE NAMES ARE GENERATED ###############
##########################################################################
## EXECUTE THIS CELL THIRD (3rd)

userInput = ""
runResults = []
print('Beginning the loop, type ''c'' to exit')
while userInput != 'c':
    choice = random.random()
    
    if(comparePosition == len(compareList)):
        # if we have exhausted the compareList, take from externalList
        sampleName = externalList[externalOrder[externalPosition]]
        externalPosition = externalPosition + 1
    elif(externalPosition == len(externalList)):
       # if we have exhausted the externalList, take from compareList    
        sampleName = compareList[compareOrder[comparePosition]]
        comparePosition = comparePosition + 1
    else:
        if(choice < biasThreshold):
        # pick from compareList
            sampleName = compareList[compareOrder[comparePosition]]
            comparePosition = comparePosition + 1
        else:
        # pick from externalList
            sampleName = externalList[externalOrder[externalPosition]]
            externalPosition = externalPosition + 1
            
    print('NAME: {}'.format(sampleName))
    userInput = input("give a rating of the name (0-10) ")
    if(userInput != 'c'):
        runResults.append([sampleName, userInput])
       
    if((comparePosition == len(compareList) and (externalPosition == len(externalList)))):
        print('exhausted both lists, exiting...')
        userInput = 'c'
        
        
# append results to the user csv
# opening the csv file in 'a+' mode 
with open(csvDir + sessionName + '.csv', 'a+', newline ='\n', encoding="utf-8") as file:     
    write = csv.writer(file) 
    write.writerows(runResults) 


In [ ]:
######################################################################
############# DATA ANALYSIS CELL, CHANGE THE PARTNER VARS ############
######################################################################

partnerA = 'partnerA.csv'
partnerB = 'partnerB.csv'

ADf = pd.read_csv(partnerA, header=None)
BDf = pd.read_csv(partnerB, header=None)
ADf = ADf.rename(columns={0:'name',1:'scoreA'})
BDf = BDf.rename(columns={0:'name',1:'scoreB'})

mergedDf = pd.merge(left=ADf, right=BDf, left_on='name', right_on='name')

mergedDf['meanAverageError'] = ((mergedDf['scoreA']**(1/2))*(mergedDf['scoreB']**(1/2)))
mergedDf.sort_values('meanAverageError',ascending=False).head(60)
